# Your Title Here

**Name(s)**: (Jake Wanderer, Calvin Truong)

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

Q1: Approximatley how much gold at 10 min is a first dragon "worth" and void grubs (in terms of winning the game)
Q2: What is more important Void grubs or First Dragon. Also hearld or 2nd dragon?
Q3: What side is more likely to win the game, and how is this related to who gets the objectives (drags,baron)
Q4: How well do heavily banned champs actually perform? (Are they best to ban?)

For now we will select question 1 to explore further.

## Step 2: Data Cleaning and Exploratory Data Analysis

In [2]:
df = pd.read_csv("Dataset/2025_LoL_esports_match_data.csv")

/var/folders/jx/n30w97zd1_75l77ctc7sskf40000gn/T/ipykernel_54338/2967102026.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Dataset/2025_LoL_esports_match_data.csv")


In [3]:
drags = ['firstdragon','dragons','opp_dragons','elementaldrakes','opp_elementaldrakes','infernals','mountains','clouds','oceans','chemtechs','hextechs','dragons (type unknown)','elders','opp_elders']

In [4]:
df

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat25,golddiffat25,xpdiffat25,csdiffat25,killsat25,assistsat25,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,LOLTMNT03_179647,complete,NaN,LFL2,2025,Winter,0,2025-01-11 11:11:24,1,15.01,...,200.0,224.0,-1.0,17.0,1.0,1.0,2.0,2.0,4.0,2.0
1,LOLTMNT03_179647,complete,NaN,LFL2,2025,Winter,0,2025-01-11 11:11:24,1,15.01,...,157.0,-2363.0,-1444.0,-18.0,0.0,1.0,2.0,1.0,7.0,0.0
2,LOLTMNT03_179647,complete,NaN,LFL2,2025,Winter,0,2025-01-11 11:11:24,1,15.01,...,241.0,-1552.0,-2465.0,-41.0,1.0,0.0,2.0,1.0,5.0,1.0
3,LOLTMNT03_179647,complete,NaN,LFL2,2025,Winter,0,2025-01-11 11:11:24,1,15.01,...,257.0,-2613.0,-1156.0,-6.0,1.0,1.0,2.0,6.0,2.0,0.0
4,LOLTMNT03_179647,complete,NaN,LFL2,2025,Winter,0,2025-01-11 11:11:24,1,15.01,...,20.0,-662.0,-734.0,18.0,0.0,2.0,2.0,0.0,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118927,LOLTMNT02_326112,complete,NaN,LAS,2025,Champ 2,1,2025-11-16 11:34:09,3,15.21,...,236.0,787.0,888.0,-23.0,9.0,4.0,1.0,4.0,3.0,1.0
118928,LOLTMNT02_326112,complete,NaN,LAS,2025,Champ 2,1,2025-11-16 11:34:09,3,15.21,...,244.0,814.0,-348.0,15.0,3.0,9.0,1.0,2.0,4.0,2.0
118929,LOLTMNT02_326112,complete,NaN,LAS,2025,Champ 2,1,2025-11-16 11:34:09,3,15.21,...,37.0,492.0,1390.0,-7.0,0.0,12.0,2.0,1.0,5.0,3.0
118930,LOLTMNT02_326112,complete,NaN,LAS,2025,Champ 2,1,2025-11-16 11:34:09,3,15.21,...,868.0,-3714.0,-6252.0,-20.0,10.0,21.0,17.0,17.0,35.0,10.0


In [5]:
df[df['gameid'] == "LOLTMNT02_326112"]['playername']

118920       Nevid
118921       Solid
118922     Despair
118923     Jinbeom
118924      Dahlia
118925    Guardian
118926     Painter
118927      SOLBON
118928      OddEye
118929      Aether
118930         NaN
118931         NaN
Name: playername, dtype: object

We see here that each game is represented by 12 columns, this seems odd at first because each game only has 10 players, but the last 2 are NaN as shown.

In [6]:
df.set_index('gameid').loc['LOLTMNT06_95435'][drags]

,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders
gameid,,,,,,,,,,,,,,
LOLTMNT06_95435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOLTMNT06_95435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOLTMNT06_95435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOLTMNT06_95435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOLTMNT06_95435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOLTMNT06_95435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOLTMNT06_95435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOLTMNT06_95435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOLTMNT06_95435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From this, we see that the dragons are null for all but the last 2 values for this game (the values where playername is NaN). So, now we know that the NaN player values represent teamwide data.

In [7]:
print(len(df)/len(df[drags][['dragons']].dropna()))

6.0


This shows us all missing values in col 'dragons' are expected because only 2 out of the 12 columns for each game should have non NaN values.

In [8]:
df['dragons'].dropna().value_counts()

dragons
2.0    4775
3.0    4628
1.0    3826
4.0    3512
0.0    2536
5.0     508
6.0      34
7.0       3
Name: count, dtype: int64

Looking at this, it seems no values should be missing, as all of these are possible.

In [9]:
drag_df = df[['dragons','elders']]

In [10]:
check_na = drag_df.isna()
df[drags][check_na['dragons'] != check_na['elders']]

,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders
58,1.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
59,0.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
70,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
71,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
94,1.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110783,0.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
110794,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
110795,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
110806,0.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


We are missing some values for the types of dragons. Because elder dragons are so different from every other type we will exclude these missing values for now.

In [11]:
drag_df = df[['dragons','elders','elementaldrakes']].dropna()

In [12]:
(drag_df['dragons'] - drag_df['elders'] == drag_df['elementaldrakes']).mean()

np.float64(1.0)

This is what we expect it means the rest of our non missing data is good.

In [13]:
elemental = df[['elementaldrakes','opp_elementaldrakes']].dropna()
elemental['counts'] = np.zeros(len(elemental))
counts = elemental.groupby(['elementaldrakes','opp_elementaldrakes']).count().reset_index()
counts['counts'] = counts['counts']/2 #Only count each game once

In [14]:
counts['pair'] = counts.apply(
    lambda row: tuple(sorted([row['elementaldrakes'], row['opp_elementaldrakes']])), axis=1
) #combine opposites
drag_counts = pd.DataFrame(counts.groupby('pair').sum()['counts']).reset_index()
drag_counts['pair'] = drag_counts['pair'].apply(lambda x: f'({str(int(x[0]))},{str(int(x[1]))})')

In [15]:
drag_counts

,pair,counts
0,"(0,0)",6.0
1,"(0,1)",32.0
2,"(0,2)",112.0
3,"(0,3)",513.0
4,"(0,4)",1772.0
5,"(1,1)",67.0
6,"(1,2)",682.0
7,"(1,3)",1446.0
8,"(1,4)",1184.0
9,"(2,2)",878.0


In [16]:
fig = px.bar(
    drag_counts,
    x='pair',
    y='counts',
    barmode='stack',
    title= "Distribution of Counts of Elemental Drake Pairings",
        labels={
        'pair': 'Pairs of number of Opponent drakes and Number of Ally drakes',
        'counts': 'Total Count'
    }
)

fig.show()

In [17]:
elemental_win = df[['elementaldrakes','opp_elementaldrakes','result']].dropna()
elemental_win = elemental_win.groupby(['elementaldrakes','opp_elementaldrakes']).mean().reset_index()
elemental_win['pair'] = elemental_win.apply(
    lambda row: tuple([row['elementaldrakes'], row['opp_elementaldrakes']]), axis=1
) #combine opposites
elemental_win['pair'] = elemental_win['pair'].apply(lambda x: f'({str(int(x[0]))},{str(int(x[1]))})')

In [18]:
elemental_win

,elementaldrakes,opp_elementaldrakes,result,pair
0,0.0,0.0,0.500000,"(0,0)"
1,0.0,1.0,0.406250,"(0,1)"
2,0.0,2.0,0.116071,"(0,2)"
3,0.0,3.0,0.042885,"(0,3)"
4,0.0,4.0,0.042325,"(0,4)"
5,1.0,0.0,0.593750,"(1,0)"
6,1.0,1.0,0.500000,"(1,1)"
7,1.0,2.0,0.326979,"(1,2)"
8,1.0,3.0,0.158368,"(1,3)"
9,1.0,4.0,0.096284,"(1,4)"


In [19]:
fig = px.bar(
    elemental_win,
    x='pair',
    y='result',
    barmode='stack',
    title= "Win Percentage for Each Pair of Number of Ally vs. Number of Opponenet Drakes",
        labels={
        'pair': 'Number of Ally Drakes Vs. Number of Opponenet Drakes',
        'result': 'Win Percentage'
    }
)

fig.show()

This graph contains redundent information e.g. win% for (0,1) = 1 - win% for (1,0). However, I think this still makes the data easiar to interpret so we will keep it.

## Step 3: Assessment of Missingness

In [20]:
# TODO

## Step 4: Hypothesis Testing

Permutation Test

H0 = Winning team gold at 10 min is equal to or less than losing team gold at 10 min.

H1 = Winning team gold at 10 min is greater than losing team gold at 10 min.

In [21]:
df = df[df['playerid'].isna()]

In [22]:
df['result'].isna().mean()

np.float64(0.0)

In [23]:
wins  = df[df['result'] == 1]['golddiffat10'].dropna()
loses = df[df['result'] == 0]['golddiffat10'].dropna()

In [24]:
win_mean = wins.mean()
loses_mean = loses.mean()
observed_diff = win_mean - loses_mean

In [25]:
df['result'].isna().mean()

np.float64(0.0)

In [26]:
non_missing = df[['result','golddiffat10']].dropna()
diffs = []
for i in range(10000):
    non_missing['result_shuffled'] = np.random.permutation(non_missing['result'])
    win_mean = non_missing[non_missing['result_shuffled'] == 1]['golddiffat10'].mean()
    lose_mean = non_missing[non_missing['result_shuffled'] == 0]['golddiffat10'].mean()
    diff = win_mean - lose_mean
    diffs.append(diff)
p_value = (diffs >= observed_diff).mean()

In [27]:
p_value

np.float64(0.0)

## Step 5: Framing a Prediction Problem

In [28]:
# TODO

We will predict who will in the game based on both gold at different points in the game, and dragons.

## Step 6: Baseline Model

In [36]:
df['firstdragon']

10        0.0
11        1.0
22        0.0
23        1.0
34        0.0
         ... 
118907    0.0
118918    1.0
118919    0.0
118930    0.0
118931    1.0
Name: firstdragon, Length: 22329, dtype: float64

In [30]:
df['goldat10']

10        14266.0
11        15988.0
22        18459.0
23        15355.0
34        16335.0
           ...   
118907    15961.0
118918    15088.0
118919    16952.0
118930    16144.0
118931    15718.0
Name: goldat10, Length: 22329, dtype: float64

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [39]:
firstdrag_gold10 = df[['firstdragon','goldat10','result']].dropna()

In [75]:
X = firstdrag_gold10[['firstdragon','goldat10']]
y = firstdrag_gold10['result']

In [76]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
pipeline = Pipeline([
    ('classifier',DecisionTreeClassifier(max_depth=5,random_state=42))
])

In [81]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('classifier',
                 DecisionTreeClassifier(max_depth=5, random_state=42))])

In [82]:
test_pred = pipeline.predict(X_test)

In [86]:
(pipeline.predict(X_train) == y_train).sum()/len(y_train)

np.float64(0.6468041237113402)

In [87]:
(test_pred == y_test).sum()/len(y_test)

np.float64(0.634414513468939)

## Step 7: Final Model

In [88]:
param_grid = {'max_depth': [3,5,7,9,11,20],
              'n_estimators': [50,100,200],
              }

In [89]:
grid_search = GridSearchCV(estimator=RandomForestClassifier(),param_grid=param_grid,cv=5)

In [90]:
grid_search.fit(X_train,y_train)

KeyboardInterrupt: 

In [72]:
grid_search.best_params_

{'max_depth': 3, 'n_estimators': 50}

In [91]:
pipeline = Pipeline([
    ('classifier',RandomForestClassifier(max_depth=3,n_estimators=50,random_state=42))
])

In [92]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('classifier',
                 RandomForestClassifier(max_depth=3, n_estimators=50,
                                        random_state=42))])

In [93]:
(pipeline.predict(X_train) == y_train).sum()/len(y_train)

np.float64(0.6449484536082474)

In [95]:
(pipeline.predict(X_test) == y_test).sum()/len(y_test)

np.float64(0.6360637713029137)

pipeline.fit(firstdrag_gold10[['firstdragon','goldat10']], firstdrag_gold10['result'])

In [23]:
# TODO

OHE dragon combinations, and avg standarized gold at 10,15,20, and 25 min.

In [24]:
# TODO

## Step 8: Fairness Analysis

In [25]:
# TODO